In [ ]:
## nessisary dependents
from glob2 import glob
import numpy as np
import math
import numpy as np
import pandas as pd
import argparse

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from utils import get_dataset_from_csv
from IPython import embed

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support


import matplotlib.pyplot as plt

In [ ]:
ARG_MODELS_PATH = "models"

In [ ]:
models = glob(ARG_MODELS_PATH + "/*")
print("Founded models:")
for i, model in enumerate(models):
    print(i, model)

In [ ]:
ARG_TEST_DATA_ORIG = "test_data_orig.csv" 
ARG_TEST_DATA_PATH = "test_data.csv" 
ARG_TRAIN_DATA_PATH = "train_data.csv" 
ARG_TRAIN_DATA_ORIG = "train_data_orig.csv" 

In [ ]:
train_data_orig = pd.read_csv(ARG_TRAIN_DATA_ORIG)
train_data = pd.read_csv(ARG_TRAIN_DATA_PATH)
test_data = pd.read_csv(ARG_TEST_DATA_ORIG)
test_data.head()

In [ ]:
CSV_HEADER = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B5_B4', 
               'B3_B2', 'B3_B4', 'B4_B5_B6', 'avw', 'B3_B5', 'diff_alg', 
               #'Area_km2', 'Shoreline_development', 'Type', 
               'bloom']

In [ ]:
batch_size = 1000

eval_dataset = get_dataset_from_csv(
    ARG_TEST_DATA_PATH, 
    CSV_HEADER,
    "bloom", batch_size=batch_size
) 

train_dataset = get_dataset_from_csv(
    ARG_TRAIN_DATA_PATH, 
    CSV_HEADER,
    "bloom", batch_size=batch_size
) 


In [ ]:
target_names = ['not-bloom', 'bloom']

In [ ]:
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score


In [ ]:
#plt.figure()
lw = 2

fpr = dict()
tpr = dict()
roc_auc = dict()

T_lists = []
model_list = []
acc_list = []
acc_train_list = []
acc_train_orig_list = []


auc_list = []
recall_list = []
pr_list = []
f1_list = []
kappa_list = []
MCC_list = []
n_param_list = []

for i, model in tqdm(enumerate(models)):
    model_name = model.split("/")[1] #model.split("/")[1].split("-")[0]
    
    
    T_list = np.linspace(0, 1, 101)
    F1_list = []
    gmean_list = []
    
    print("eval model: {}".format(model))
    m0 = keras.models.load_model(model)
    n_param = m0.count_params()
    
    yhat_train = m0.predict(train_dataset)
    y_true_train = train_data["bloom"]
    


    for T in T_list:
        y_pred = yhat_train >= T
        y_true = y_true_train

        f1 = f1_score(y_true, y_pred)


        F1_list.append(f1)

    T = T_list[np.argmax(F1_list)]
    T_lists.append(T)
    
    

    
    
    y_pred_train = (yhat_train > T).astype(np.int)
    ac_train = accuracy_score(y_true_train, y_pred_train)
    acc_train_list.append(ac_train)


    
    yhat = m0.predict(eval_dataset)
    y_true = test_data["bloom"]
    y_pred = (yhat > T).astype(np.int)
    
    ac = accuracy_score(y_true, y_pred)
    
    
    y_pred_train = (yhat > 0.5).astype(np.int)
    ac_train = accuracy_score(y_true, y_pred_train)
    acc_train_orig_list.append(ac_train)
    
    
    fpr, tpr, thresholds = metrics.roc_curve(y_true, yhat)
        
    auc = metrics.auc(fpr, tpr)
    rec = recall_score(y_true, y_pred)
    pre = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    kappa = cohen_kappa_score(y_true, y_pred)
    mc = matthews_corrcoef(y_true, y_pred)
    
    model_list.append(model_name)
    acc_list.append(ac)
    
    auc_list.append(auc)
    recall_list.append(rec)
    pr_list.append(pre)
    f1_list.append(f1)
    kappa_list.append(kappa)
    MCC_list.append(mc)
    n_param_list.append(n_param)


In [ ]:
model.split("/")[1]#.split("-")[0]

In [ ]:
idx = np.argmax(acc_list)
idx

In [ ]:
summary_table = pd.DataFrame({"Model": model_list, 
                              "T": T_lists,
                              "Accuracy": acc_list,
                              "Accuracy train": acc_train_list,
                              "Accuracy orig": acc_train_orig_list,
                              "Number of params": n_param_list,
                            })
summary_table = summary_table.sort_values(by=['Accuracy'], ascending=False)
summary_table

In [ ]:
print(summary_table.round(3).to_latex(index=False))

In [ ]:
test_results = pd.DataFrame({"Model": model_list, 
                             "Accuracy": acc_list,
                             "AUC":auc_list,
                             "Recall":recall_list,
                             "Prec.":pr_list,
                             "F1":f1_list,
                             "Kappa":kappa_list,
                             "MCC":MCC_list,
                             #"TT (Sec)":0.0
                             "T": T_lists,
                             "Accuracy (train)": acc_train_list,
                             "Number of params": n_param_list,
                            })

test_results

In [ ]:
test_results = test_results.sort_values(by=['Accuracy'], ascending=False)
test_results

In [ ]:
test_results = test_results.sort_values(by=['AUC'], ascending=False)
test_results

In [ ]:
# test_results = test_results.sort_values(by=['Prec.'], ascending=False)
test_results

In [ ]:
import pycaret
from pycaret.classification import *

In [ ]:
exp_clf101 = setup(data = train_data, target = 'bloom', 
                   test_data = test_data,
                   remove_multicollinearity = True, multicollinearity_threshold = 0.4,
                   feature_selection = True,
                   feature_interaction = True,
                   feature_ratio = True,
                   session_id=123) 

In [ ]:
best_model = compare_models(n_select = 15)

In [ ]:
#save_model(best_model, 'best_models')

In [ ]:
res_list = []
train_acc = []
for j in range(15):
    predictions = predict_model(best_model[j], data=test_data)
    res = pull()
    res_list.append(res)
    predictions = predict_model(best_model[j], data=train_data)
    res = pull()
    train_acc.append(float(res["Accuracy"]))

In [ ]:
pycaret= pd.concat(res_list)
pycaret["Accuracy (train)"] = train_acc
pycaret["Number of params"] = "-"
pycaret

In [ ]:
list(pycaret["Model"])

In [ ]:
final = pd.concat([test_results, pycaret])

In [ ]:
final = final.sort_values(by=['Accuracy'], ascending=False)
final